# LSTM. 8 featerow - mag, x, y, depth, dist, distance, plate, days_diff

In [1]:
import numpy as np
import tensorflow as tf
import pandas as pd
SEED = 1337
tf.random.set_seed(SEED)
np.random.seed(SEED)

2024-01-17 20:39:51.068532: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-01-17 20:39:51.068629: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-01-17 20:39:51.072199: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-01-17 20:39:51.366012: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-01-17 20:39:53.575470: W tensorflow/compiler/

In [2]:
ds_train = tf.data.Dataset.load("../data/ds_train_3/")
ds_val = tf.data.Dataset.load("../data/ds_val_3/")
ds_test = tf.data.Dataset.load("../data/ds_test_3/")

2024-01-17 20:39:58.046502: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:880] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-01-17 20:39:58.412770: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:880] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-01-17 20:39:58.412908: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:880] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-01-17 20:39:58.417575: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:880] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-01-17 20:39:58.417696: I tensorflow/compile

In [3]:
ds_train.cardinality().numpy(), ds_val.cardinality().numpy(), ds_test.cardinality().numpy()

(3385278, 488322, 106261)

In [4]:
BATCH_SIZE = 1024
BLOCK_SIZE = 32

In [5]:
def fix_x_and_y(features, context, labels):
    return features, labels

In [6]:
ds_train = ds_train.map(fix_x_and_y)
ds_val = ds_val.map(fix_x_and_y)
ds_test = ds_test.map(fix_x_and_y)

In [7]:
for x, y in ds_train.batch(BATCH_SIZE).take(1):
    print(x.shape)
    print(x[:,:,:-3].shape, x[:,:,-3].shape, x[:,:,-2].shape, x[:,:,-1].shape)
    print(y.shape)

(1024, 32, 9)
(1024, 32, 6) (1024, 32) (1024, 32) (1024, 32)
(1024, 1)


In [8]:
ys = []
for x, y in ds_train.batch(BATCH_SIZE):
    ys.append(y.numpy())
ys = np.concatenate(ys)
np.bincount(tf.cast(ys, tf.int32).numpy().flatten())

array([2992310,  392968])

In [9]:
#neg, pos = np.bincount(tf.cast(ys, tf.int32).numpy().flatten())
neg, pos = 2992310, 392968
total = neg + pos
total, pos, neg

(3385278, 392968, 2992310)

In [10]:
ds_train = ds_train.batch(BATCH_SIZE).shuffle(2000, seed=SEED).prefetch(tf.data.experimental.AUTOTUNE)
ds_val = ds_val.batch(BATCH_SIZE).prefetch(tf.data.experimental.AUTOTUNE)
ds_test = ds_test.batch(BATCH_SIZE).prefetch(tf.data.experimental.AUTOTUNE)

In [11]:
@tf.keras.saving.register_keras_serializable()
class MyModel(tf.keras.Model):
    def __init__(self, n_embed):
        super(MyModel, self).__init__()
        self.embed_plate = tf.keras.layers.Embedding(32, n_embed // 9, name="embed_plate")
        self.embed_dd = tf.keras.layers.Embedding(20, n_embed // 9, name="embed_dd")
        self.embed_magtype = tf.keras.layers.Embedding(20, n_embed // 9, name="embed_magtype")
        self.embed = tf.keras.layers.Dense(n_embed // 9 * 6, name="embed_continous")
        self.lstm1 = tf.keras.layers.LSTM(n_embed, return_sequences=True, name="lstm1")
        self.lstm2 = tf.keras.layers.LSTM(n_embed, name="lstm2")
        self.dense1 = tf.keras.layers.Dense(n_embed, activation='relu', name="pre_classifier")
        self.dense2 = tf.keras.layers.Dense(1, activation='sigmoid', name="classifier")
        self.conc = tf.keras.layers.Concatenate(axis=-1, name="concatenate")

    def build_graph(self):
        x = [tf.keras.layers.Input(shape=(BLOCK_SIZE, 6), batch_size=None, name="input_continous"),
            tf.keras.layers.Input(shape=(BLOCK_SIZE), batch_size=None, name="input_plate"),
            tf.keras.layers.Input(shape=(BLOCK_SIZE), batch_size=None, name="input_dd"),
            tf.keras.layers.Input(shape=(BLOCK_SIZE), batch_size=None, name="input_magtype")]
        return tf.keras.Model(inputs=x, outputs=self.call(x))

    def call(self, x):
        cont, plate, dd, magtype = x[:,:,:-3], x[:,:,-3], x[:,:,-2], x[:,:,-1]
        x1 = self.embed(cont)
        x2 = self.embed_plate(plate)
        x3 = self.embed_dd(dd)
        x4 = self.embed_magtype(magtype)
        x = self.conc([x1, x2, x3, x4])
        x = self.lstm1(x)
        x = self.lstm2(x)
        x = self.dense1(x)
        x = self.dense2(x)
        return x

In [12]:
model = MyModel(135)

In [13]:
model.build(input_shape=(None, BLOCK_SIZE, 9))
model.count_params()

312886

In [14]:
model.compile(loss=tf.keras.losses.BinaryCrossentropy(from_logits=False),
            optimizer='adam',
            metrics=['accuracy', tf.keras.metrics.Recall(), tf.keras.metrics.Precision()])

In [15]:
weight_for_0 = (1 / neg) * (total / 2.0)
weight_for_1 = (1 / pos) * (total / 2.0)
class_weight = {0: weight_for_0, 1: weight_for_1}
history = model.fit(ds_train, epochs=20, validation_data=ds_val, class_weight=class_weight)

Epoch 1/20


2024-01-17 20:42:44.829626: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:422] ShuffleDatasetV3:20: Filling up shuffle buffer (this may take a while): 229 of 2000
2024-01-17 20:43:04.824129: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:422] ShuffleDatasetV3:20: Filling up shuffle buffer (this may take a while): 698 of 2000
2024-01-17 20:43:24.805701: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:422] ShuffleDatasetV3:20: Filling up shuffle buffer (this may take a while): 1148 of 2000
2024-01-17 20:43:34.807196: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:422] ShuffleDatasetV3:20: Filling up shuffle buffer (this may take a while): 1385 of 2000
2024-01-17 20:43:44.830037: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:422] ShuffleDatasetV3:20: Filling up shuffle buffer (this may take a while): 1630 of 2000
2024-01-17 20:43:59.556294: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:422] ShuffleDatasetV3:20: Filling up shuffle buffer (this may take

3306/3306 [==============================] - 506s 120ms/step - loss: 0.5295 - accuracy: 0.7979 - recall: 0.6579 - precision: 0.3198 - val_loss: 0.4695 - val_accuracy: 0.8312 - val_recall: 0.4092 - val_precision: 0.3934
Epoch 2/20


2024-01-17 20:51:06.875393: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:422] ShuffleDatasetV3:20: Filling up shuffle buffer (this may take a while): 189 of 2000
2024-01-17 20:51:26.839243: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:422] ShuffleDatasetV3:20: Filling up shuffle buffer (this may take a while): 557 of 2000
2024-01-17 20:51:36.884434: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:422] ShuffleDatasetV3:20: Filling up shuffle buffer (this may take a while): 745 of 2000
2024-01-17 20:51:56.870401: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:422] ShuffleDatasetV3:20: Filling up shuffle buffer (this may take a while): 1107 of 2000
2024-01-17 20:52:06.871306: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:422] ShuffleDatasetV3:20: Filling up shuffle buffer (this may take a while): 1275 of 2000
2024-01-17 20:52:26.848334: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:422] ShuffleDatasetV3:20: Filling up shuffle buffer (this may take 

3306/3306 [==============================] - 599s 148ms/step - loss: 0.4881 - accuracy: 0.8256 - recall: 0.6840 - precision: 0.3656 - val_loss: 0.4729 - val_accuracy: 0.8339 - val_recall: 0.4459 - val_precision: 0.4077
Epoch 3/20


...

3306/3306 [==============================] - 656s 171ms/step - loss: 0.2318 - accuracy: 0.8792 - recall: 0.9276 - precision: 0.4894 - val_loss: 0.8772 - val_accuracy: 0.7668 - val_recall: 0.4740 - val_precision: 0.2898
Epoch 19/20


2024-01-17 23:54:12.872786: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:422] ShuffleDatasetV3:20: Filling up shuffle buffer (this may take a while): 196 of 2000
2024-01-17 23:54:32.858325: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:422] ShuffleDatasetV3:20: Filling up shuffle buffer (this may take a while): 585 of 2000
2024-01-17 23:54:52.873459: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:422] ShuffleDatasetV3:20: Filling up shuffle buffer (this may take a while): 1024 of 2000
2024-01-17 23:55:12.857875: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:422] ShuffleDatasetV3:20: Filling up shuffle buffer (this may take a while): 1471 of 2000
2024-01-17 23:55:32.848012: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:422] ShuffleDatasetV3:20: Filling up shuffle buffer (this may take a while): 1919 of 2000
2024-01-17 23:55:36.419751: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:452] Shuffle buffer filled.


3306/3306 [==============================] - 656s 170ms/step - loss: 0.2182 - accuracy: 0.8857 - recall: 0.9356 - precision: 0.5041 - val_loss: 0.9841 - val_accuracy: 0.7878 - val_recall: 0.4481 - val_precision: 0.3130
Epoch 20/20


2024-01-18 00:05:09.276372: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:422] ShuffleDatasetV3:20: Filling up shuffle buffer (this may take a while): 188 of 2000
2024-01-18 00:05:29.241718: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:422] ShuffleDatasetV3:20: Filling up shuffle buffer (this may take a while): 565 of 2000
2024-01-18 00:05:39.242466: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:422] ShuffleDatasetV3:20: Filling up shuffle buffer (this may take a while): 784 of 2000
2024-01-18 00:05:49.263439: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:422] ShuffleDatasetV3:20: Filling up shuffle buffer (this may take a while): 997 of 2000
2024-01-18 00:06:09.263353: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:422] ShuffleDatasetV3:20: Filling up shuffle buffer (this may take a while): 1453 of 2000
2024-01-18 00:06:19.269283: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:422] ShuffleDatasetV3:20: Filling up shuffle buffer (this may take a

3306/3306 [==============================] - 652s 169ms/step - loss: 0.2050 - accuracy: 0.8919 - recall: 0.9424 - precision: 0.5189 - val_loss: 0.9912 - val_accuracy: 0.7880 - val_recall: 0.4584 - val_precision: 0.3159


In [16]:
model.evaluate(ds_test)

104/104 [==============================] - 11s 106ms/step - loss: 0.8765 - accuracy: 0.8043 - recall: 0.4447 - precision: 0.2503


[0.8764833807945251,
 0.8042743802070618,
 0.4446864426136017,
 0.2502681314945221]

In [17]:
model.save("../models/model_v7_cw.keras")

In [18]:
# save history
history = pd.DataFrame.from_dict(history.history)
history.to_csv("../models/history_model_v7_cw.csv", index=False)